In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from tensorflow import keras

In [2]:
df = pd.read_csv("./wavefront_reconstruction_with_artificial.csv")
df.head()

,id,incoming_wavefront_x,incoming_wavefront_y,output_x,output_y,avg_rms_error
0,1,-0.97708,0.03072,0.00167,0.000011,0.045974
1,2,-0.97360,-0.06114,0.00370,0.000013,0.020102
2,3,-0.96199,-0.09952,0.00662,0.000006,0.032828
3,4,-0.95445,-0.03440,0.01025,0.000009,0.068953
4,5,-0.95445,0.10340,0.01564,0.000017,0.137871


In [3]:
df['avg_rms_error'].value_counts()

avg_rms_error
0.246824    2
0.283160    2
0.045974    1
0.325318    1
0.041868    1
           ..
0.684931    1
0.504976    1
0.510791    1
0.053546    1
0.000023    1
Name: count, Length: 1706, dtype: int64

In [4]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['avg_rms_error'] = le.fit_transform(df['avg_rms_error'])
df.head()

,id,incoming_wavefront_x,incoming_wavefront_y,output_x,output_y,avg_rms_error
0,1,-0.97708,0.03072,0.00167,0.000011,271
1,2,-0.97360,-0.06114,0.00370,0.000013,118
2,3,-0.96199,-0.09952,0.00662,0.000006,196
3,4,-0.95445,-0.03440,0.01025,0.000009,400
4,5,-0.95445,0.10340,0.01564,0.000017,691


In [5]:
affecting_parameter_name = le.classes_

In [6]:
X = df.drop(columns=['id', 'avg_rms_error'])
y = df['avg_rms_error']
X.head(3)

,incoming_wavefront_x,incoming_wavefront_y,output_x,output_y
0,-0.97708,0.03072,0.00167,0.000011
1,-0.97360,-0.06114,0.00370,0.000013
2,-0.96199,-0.09952,0.00662,0.000006


In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=69)

In [9]:
x_train.shape

(1195, 4)

In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
print(x_train[:1])
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)
print(x_train[:1])
print(x_test[:1])

     incoming_wavefront_x  incoming_wavefront_y  output_x  output_y
869              -0.08705               0.88368   0.02889  0.000026
[[ 0.01135137  1.4877317  -1.10043548 -1.23662872]]
[[-0.21264496  1.5532081  -0.21206042 -0.46202608]]


In [12]:
y_train = keras.utils.to_categorical(y_train, num_classes=1706)
print(y_train[:5])

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [13]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [17]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(x_train.shape[-1], )))
model.add(Dense(units=32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=1706, activation='softmax'))

In [18]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 32)                160       
                                                                 
 dense_4 (Dense)             (None, 32)                1056      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense_5 (Dense)             (None, 1706)              56298     
                                                                 
Total params: 57514 (224.66 KB)
Trainable params: 57514 (224.66 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
model.fit(x_train, y_train, epochs=100, verbose=2)

Epoch 1/100
38/38 - 1s - loss: 7.4579 - accuracy: 0.0000e+00 - 675ms/epoch - 18ms/step
Epoch 2/100
38/38 - 0s - loss: 7.3928 - accuracy: 0.0042 - 59ms/epoch - 2ms/step
Epoch 3/100
38/38 - 0s - loss: 7.2519 - accuracy: 0.0017 - 60ms/epoch - 2ms/step
Epoch 4/100
38/38 - 0s - loss: 6.9193 - accuracy: 0.0059 - 59ms/epoch - 2ms/step
Epoch 5/100
38/38 - 0s - loss: 6.4771 - accuracy: 0.0067 - 59ms/epoch - 2ms/step
Epoch 6/100
38/38 - 0s - loss: 6.1804 - accuracy: 0.0042 - 60ms/epoch - 2ms/step
Epoch 7/100
38/38 - 0s - loss: 5.9811 - accuracy: 0.0134 - 58ms/epoch - 2ms/step
Epoch 8/100
38/38 - 0s - loss: 5.7640 - accuracy: 0.0100 - 61ms/epoch - 2ms/step
Epoch 9/100
38/38 - 0s - loss: 5.6263 - accuracy: 0.0201 - 58ms/epoch - 2ms/step
Epoch 10/100
38/38 - 0s - loss: 5.5603 - accuracy: 0.0067 - 61ms/epoch - 2ms/step
Epoch 11/100
38/38 - 0s - loss: 5.4372 - accuracy: 0.0159 - 59ms/epoch - 2ms/step
Epoch 12/100
38/38 - 0s - loss: 5.4423 - accuracy: 0.0151 - 61ms/epoch - 2ms/step
Epoch 13/100
38/38 

In [20]:
prediction = model.predict(x_test)

17/17 [==============================] - 0s 1ms/step


In [22]:
model.save('laser_beam_model.h5', model)

c:\Users\ashen\anaconda3\envs\detection\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
from tensorflow import keras
from keras.models import load_model
df = pd.read_csv("./wavefront_reconstruction_with_artificial.csv")
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
le = LabelEncoder()
df['avg_rms_error'] = le.fit_transform(df['avg_rms_error'])
X = df.drop(columns=['id', 'avg_rms_error'])
y = df['avg_rms_error']
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=69)
x_train = sc.fit_transform(x_train)
model = load_model('laser_beam_model.h5')
affecting_parameter_name = le.classes_


# Initialize an empty list to store the captured values
captured_values = []

# Define a while loop that continues indefinitely
while True:
    # Prompt the user for input
    message = input("Enter a value (or type 'done' to finish): ")

    # Check if the user wants to exit the loop
    if message.lower() == 'done':
        break  # Exit the loop if the user types 'done'

    # Convert the input value to the appropriate data type and append it to the list
    try:
        value = float(message)  # Convert input to float
        captured_values.append(value)
    except ValueError:
        print("Invalid input. Please enter a valid number.")

# Ensure that exactly four values were captured
if len(captured_values) == 4:  # Check if exactly five values were captured
    # Assign the captured values to data1, data2, data3, and data4
    data1, data2, data3, data4 = captured_values
    X_new = np.array([[data1 , data2, data3, data4 ]])
    x_test = sc.transform(X_new)
    prediction = model.predict(x_test)
    prediction = np.argmax(prediction, axis=-1)
    error = (affecting_parameter_name[prediction[:1][0]]/0.2853)*14
    print("Most Affecting Error:",error,"%")
    # x1_test = sc.transform([[data1, data2, data3, data4, data5]])
    
    # Use the model to make predictions
    # pred = model.predict([[data1, data2, data3, data4, data5]])
    # pred_name = np.argmax(pred, axis=-1)
    
    # Print the predictions
    # print([[data1, data2, data3, data4, data5]].shape())
    # print("Predictions:", pred_name)
else:
    print("Please enter exactly five values.")


1/1 [==============================] - 0s 37ms/step
Most Affecting Error: 13.247066456361724 %


C:\Users\ashen\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
